In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 4 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

14 November 2022 ; 21:02:05


# Features, model

In [3]:
features = ['VORP','WS','OWS','BPM','PER','PTS','FT','FTA','OBPM','FG','USG%','Seed','MP']#,'GS'] # GS !!!
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/mvp/mvp_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') 

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,Precious Achiuwa,C,23,TOR,12,0,20.4,3.0,7.7,0.391,...,2.36,114.37,112.28,2.09,3.24,114.9,111.9,3.0,0,1
1,OG Anunoby,SF,25,TOR,14,14,35.4,6.4,13.6,0.474,...,2.36,114.37,112.28,2.09,3.24,114.9,111.9,3.0,0,0
2,Dalano Banton,PG,23,TOR,13,0,9.5,1.8,4.3,0.429,...,2.36,114.37,112.28,2.09,3.24,114.9,111.9,3.0,0,1
3,Scottie Barnes,PF,21,TOR,13,13,31.9,5.5,12.8,0.431,...,2.36,114.37,112.28,2.09,3.24,114.9,111.9,3.0,0,0
4,Khem Birch,C,30,TOR,4,0,8.5,0.8,1.3,0.600,...,2.36,114.37,112.28,2.09,3.24,114.9,111.9,3.0,0,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] >= season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'Share']]
res.head()

,Player,Tm,PER,VORP,WS,PTS,Seed,Share
0,Giannis Antetokounmpo,MIL,32.1,9.111111,15.488889,31.8,1,0.895821
1,Luka Dončić,DAL,33.3,10.933333,18.450000,34.3,6,0.738123
2,Nikola Jokić,DEN,29.2,8.200000,15.138462,20.8,1,0.668791
3,Stephen Curry,GSW,29.8,9.566667,16.400000,32.8,13,0.559835
4,Jayson Tatum,BOS,27.7,7.569231,15.138462,32.3,2,0.532064


In [8]:
res.to_csv(path + f'Results/mvp/results_week_{week}.csv', index = None)